In [4]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import csv
import os
import shutil

# Initialize list to store uploaded file paths
uploaded_files = []

# Define the base directory dynamically based on the current working directory
base_dir = os.getcwd()  # Use current working directory as base
applications_dir = os.path.join(base_dir, 'Application Forms')

# Function to handle form submission
def submit_form():
    applicant_name = entry_applicant_name.get()
    applicant_address = entry_applicant_address.get()
    inventor_name = entry_inventor_name.get()
    inventor_contact = entry_inventor_contact.get()
    title = entry_title.get().strip()
    abstract = text_abstract.get("1.0", tk.END).strip()
    claims = [claim.get() for claim in claims_entries]
    description = text_description.get("1.0", tk.END).strip()
    field_of_invention_value = field_of_invention.get()
    if field_of_invention_value == "Other":
        field_of_invention_value = custom_field_entry.get()  # Get custom value
    declaration_checked = declaration_var.get()

    # Check if all required fields are filled
    if not (applicant_name and applicant_address and inventor_name and inventor_contact and title and abstract and description and field_of_invention_value and declaration_checked):
        messagebox.showwarning("Incomplete Data", "Please fill out all required fields before submitting.")
        return

    # Create a folder based on the Patent Title in the dynamic base directory
    save_path = os.path.join(applications_dir, title)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Save data to CSV named after the patent title in the specified directory
    csv_file_path = os.path.join(save_path, f'{title}.csv')
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["applicant_name", "applicant_address", "inventor_name", "inventor_contact", "title", "abstract", "claims", "description", "field_of_invention", "declaration_of_inventorship"])
        writer.writerow([applicant_name, applicant_address, inventor_name, inventor_contact, title, abstract, " | ".join(claims), description, field_of_invention_value, declaration_checked])

    # Save uploaded drawings to the folder
    for drawing_path in uploaded_files:
        if os.path.isfile(drawing_path):
            shutil.copy(drawing_path, save_path)

    # Show success message and close the application
    messagebox.showinfo("Success", "Patent Application Submitted Successfully!")
    root.destroy()  # Close the application window

# Function to handle form cancellation
def cancel_form():
    root.destroy()

# Function to handle file uploads
def upload_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("PDF files", "*.pdf"), ("JPEG files", "*.jpeg"), ("All files", "*.*")])
    for file_path in file_paths:
        if file_path:
            uploaded_files.append(file_path)
            row_index = len(uploaded_files) + 15
            file_label = ttk.Label(scrollable_frame, text=file_path, background='white', foreground='green')
            file_label.grid(row=row_index, column=1, sticky='w', pady=2, padx=10)
            remove_button = ttk.Button(scrollable_frame, text="X", command=lambda fl=file_label, fp=file_path, rb=row_index: remove_file(fl, fp, rb))
            remove_button.grid(row=row_index, column=2, padx=5)

# Function to remove uploaded file
def remove_file(file_label, file_path, row_index):
    if file_path in uploaded_files:
        uploaded_files.remove(file_path)
    file_label.destroy()
    for widget in scrollable_frame.grid_slaves():
        if int(widget.grid_info()["row"]) == row_index:
            widget.destroy()
    reposition_fields()

# Function to reposition fields after changes
def reposition_fields():    
    current_row = 11 + len(claims_entries)

    # Move the "Add Claim" button below the last claim entry
    add_claim_button.grid(row=current_row, column=1, pady=5, padx=10, sticky='w')
    current_row += 1

    text_description_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    text_description.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    drawings_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    button_upload_drawings.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    field_of_invention_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    field_of_invention.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    if field_of_invention.get() == "Other":
        custom_field_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
        custom_field_entry.grid(row=current_row, column=1, pady=5, padx=10)
        current_row += 1
    else:
        custom_field_label.grid_forget()
        custom_field_entry.grid_forget()

    text_background_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    text_background.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    text_pct_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    text_pct.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    declaration_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    declaration_checkbox.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    text_related_applications_label.grid(row=current_row, column=0, sticky='w', pady=5, padx=10)
    text_related_applications.grid(row=current_row, column=1, pady=5, padx=10)
    current_row += 1

    button_frame.grid(row=current_row, column=1, pady=20)

# Function to dynamically add new claim fields with numbering
def add_claim_field():
    claim_number = len(claims_entries) + 1  # Get the next claim number
    claim_label = ttk.Label(scrollable_frame, text=f"Claim {claim_number}:", background='white')
    claim_label.grid(row=11 + len(claims_entries), column=0, pady=2, padx=10, sticky='w')

    claim_entry = ttk.Entry(scrollable_frame, width=50)
    claims_entries.append(claim_entry)
    claim_entry.grid(row=11 + len(claims_entries) - 1, column=1, pady=2, padx=10)
    
    reposition_fields()

# Function to dynamically show or hide the custom field when "Other" is selected in "Field of Invention"
def check_field_of_invention(event):
    reposition_fields()

# Create main window
root = tk.Tk()
root.title("Patent Application Submission Form")
root.geometry("900x650")

root.configure(bg='white')

# Create a scrollable canvas
main_frame = tk.Frame(root, bg='white')
main_frame.pack(fill='both', expand=True, padx=20, pady=20)
canvas = tk.Canvas(main_frame, bg='white')
scrollbar = ttk.Scrollbar(main_frame, orient='vertical', command=canvas.yview)
scrollable_frame = tk.Frame(canvas, bg='white')

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
canvas.configure(yscrollcommand=scrollbar.set)

canvas.pack(side='left', fill='both', expand=True)
scrollbar.pack(side='right', fill='y')

# Enable mouse scrolling
canvas.bind_all("<MouseWheel>", lambda event: canvas.yview_scroll(-1 * (event.delta // 120), "units"))

# Header Label
header_frame = tk.Frame(scrollable_frame, bg='#007acc', padx=10, pady=10)
header_frame.grid(row=1, column=0, columnspan=2, pady=20, sticky='ew')
tk.Label(header_frame, text="European Patent Office - Patent Application Submission Form", font=("Helvetica", 16, "bold"), bg='#007acc', fg='white').pack()

# Section Headers
def create_section_header(frame, text, row):
    tk.Label(frame, text=text, font=("Helvetica", 14, "bold"), bg='white', fg='#007acc').grid(row=row, column=0, columnspan=2, pady=10, sticky='w')

# Function to create labeled entry
def create_labeled_entry(frame, label_text, row, required=True):
    ttk.Label(frame, text=f"{label_text}{' (Required):' if required else ':'}", background='white').grid(row=row, column=0, sticky='w', pady=5, padx=10)
    entry = ttk.Entry(frame, width=50)
    entry.grid(row=row, column=1, pady=5, padx=10)
    return entry

# Personal Information Section
create_section_header(scrollable_frame, "Personal Information", 2)
entry_applicant_name = create_labeled_entry(scrollable_frame, "Applicant Name", 3)
entry_applicant_address = create_labeled_entry(scrollable_frame, "Applicant Address", 4)

# Inventor Information Section
create_section_header(scrollable_frame, "Inventor Information", 5)
entry_inventor_name = create_labeled_entry(scrollable_frame, "Inventor Name", 6)
entry_inventor_contact = create_labeled_entry(scrollable_frame, "Inventor Contact Details", 7)

# Patent Information Section
create_section_header(scrollable_frame, "Patent Information", 8)
entry_title = create_labeled_entry(scrollable_frame, "Patent Title", 9)

# Abstract
tk.Label(scrollable_frame, text="Abstract (Max 250 words, Required):", bg='white').grid(row=10, column=0, sticky='w', pady=5, padx=10)
text_abstract = tk.Text(scrollable_frame, width=60, height=5, font=("Helvetica", 12))
text_abstract.grid(row=10, column=1, pady=5, padx=10)

# Claims
tk.Label(scrollable_frame, text="Claims (Required):", bg='white').grid(row=11, column=0, sticky='w', pady=5, padx=10)
claims_entries = [ttk.Entry(scrollable_frame, width=50) for _ in range(3)]
for i, claim_entry in enumerate(claims_entries):
    ttk.Label(scrollable_frame, text=f"Claim {i + 1}:", background='white').grid(row=11 + i, column=0, pady=2, padx=10, sticky='w')
    claim_entry.grid(row=11 + i, column=1, pady=2, padx=10)

# "Add Claim" Button
add_claim_button = ttk.Button(scrollable_frame, text="Add Claim", command=add_claim_field)
add_claim_button.grid(row=14, column=1, pady=5, padx=10, sticky='w')

# Description
text_description_label = tk.Label(scrollable_frame, text="Detailed Description or Specification (Required):", bg='white')
text_description_label.grid(row=15, column=0, sticky='w', pady=5, padx=10)
text_description = tk.Text(scrollable_frame, width=60, height=10, font=("Helvetica", 12))
text_description.grid(row=15, column=1, pady=5, padx=10)

# Drawings
drawings_label = tk.Label(scrollable_frame, text="Drawings (Upload PDF, JPEG, etc.):", bg='white')
drawings_label.grid(row=16, column=0, sticky='w', pady=5, padx=10)
button_upload_drawings = ttk.Button(scrollable_frame, text="Upload Drawings", command=upload_files)
button_upload_drawings.grid(row=16, column=1, pady=5, padx=10)

# Field of Invention/Technical Area
field_of_invention_label = tk.Label(scrollable_frame, text="Field of Invention/Technical Area (Required):", bg='white')
field_of_invention_label.grid(row=17, column=0, sticky='w', pady=5, padx=10)
field_of_invention = ttk.Combobox(scrollable_frame, values=["Biotechnology", "Chemistry", "Electronics", "Mechanical", "Pharmaceuticals", "Software", "Other"])
field_of_invention.grid(row=17, column=1, pady=5, padx=10)
field_of_invention.bind("<<ComboboxSelected>>", check_field_of_invention)

# Custom field for "Other"
custom_field_label = tk.Label(scrollable_frame, text="Please specify the field of invention:", bg='white')
custom_field_entry = ttk.Entry(scrollable_frame, width=50)

# Background Information
text_background_label = tk.Label(scrollable_frame, text="Background Information:", bg='white')
text_background_label.grid(row=18, column=0, sticky='w', pady=5, padx=10)
text_background = tk.Text(scrollable_frame, width=60, height=3, font=("Helvetica", 12))
text_background.grid(row=18, column=1, pady=5, padx=10)

# PCT Information
text_pct_label = tk.Label(scrollable_frame, text="Patent Cooperation Treaty (PCT) Information (Optional):", bg='white')
text_pct_label.grid(row=19, column=0, sticky='w', pady=5, padx=10)
text_pct = tk.Text(scrollable_frame, width=60, height=3, font=("Helvetica", 12))
text_pct.grid(row=19, column=1, pady=5, padx=10)

# Declaration of Inventorship
declaration_label = tk.Label(scrollable_frame, text="Declaration of Inventorship (Required):", bg='white')
declaration_label.grid(row=21, column=0, sticky='w', pady=5, padx=10)
declaration_var = tk.BooleanVar()
declaration_checkbox = ttk.Checkbutton(scrollable_frame, text="I declare that I am the inventor or authorized representative.", variable=declaration_var)
declaration_checkbox.grid(row=21, column=1, pady=5, padx=10)

# Disclosure of Any Related Applications
text_related_applications_label = tk.Label(scrollable_frame, text="Disclosure of Any Related Applications:", bg='white')
text_related_applications_label.grid(row=22, column=0, sticky='w', pady=5, padx=10)
text_related_applications = tk.Text(scrollable_frame, width=60, height=3, font=("Helvetica", 12))
text_related_applications.grid(row=22, column=1, pady=5, padx=10)

# Submit and Cancel Buttons
button_frame = tk.Frame(scrollable_frame, bg='white')
button_frame.grid(row=23, column=1, pady=20)

submit_button = ttk.Button(button_frame, text="Submit Application", command=submit_form)
submit_button.grid(row=0, column=0, padx=10)

cancel_button = ttk.Button(button_frame, text="Cancel", command=cancel_form)
cancel_button.grid(row=0, column=1, padx=10)

# Run the main loop
root.mainloop()
